In [2]:
import nltk
import random
import os
from sklearn import svm
import json
import random
import pickle
import numpy as np

In [3]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [4]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[each_file]) for sentence in data_temp]
    return dataset

In [15]:
def to_bigram(data):
    dataset=[]
    dataset+=[([items for items in nltk.bigrams(sentence.split())],polarity) for sentence,polarity in data]    
    return dataset        

In [26]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
with open('../NaiveBayes/vocab_bigram') as f:
    vocab=json.load(f)    

In [27]:
#bigrams
print labeled_data[:5]

[(u'work team qualiti', 2), (u'keep van good shape', 4), (u'great place work mold career path', 4), (u'learn manag compani', 2), (u'work center', 1)]


In [28]:
print len(labeled_data)

201378


In [29]:
print len(vocab)
print vocab[:100]

10863
[u'team set', u'still get', u'work abl', u'logist team', u'good reput', u'year employ', u'time realli', u'quickli learn', u'custom experi', u'steep learn', u'perform well', u'drive sale', u'horribl work', u'languag easi', u'anywher els', u'time peopl', u'hard move', u'sinc start', u'realli smart', u'learn thing', u'work alway', u'lot challeng', u'offer great', u'import part', u'meet amp', u'style manag', u'friendli manner', u'job face', u'job fact', u'life properli', u'work field', u'proud inner', u'provid report', u'target good', u'learn best', u'day work', u'interest client', u'custom manag', u'fantast learn', u'though good', u'item anyth', u'posit allow', u'clean work', u'gave opportun', u'learn comput', u'six month', u'great like', u'larg tear', u'let us', u'given us', u'work bore', u'work form', u'work forc', u'also pay', u'lo due', u'month experi', u'time listen', u'kind environ', u'tide work', u'adventur brilliant', u'go life', u'learn intern', u'otherwis good', u'togeth a

In [36]:
def extract_features(dataset):       #input a  sentence ...output bigrams
    feature_vector=[]
    labels=[]
    for sentence,label in dataset:
        features = {}
        labels.append(label)
        sentence=[items[0]+' '+items[1] for items in nltk.bigrams(sentence.split())]
        for word in vocab:
            features[word] = 0
            if word in sentence:
                features[word] = 1
        feature_vector.append(features.values())        
    return {'feature':feature_vector,'labels':labels}        

In [37]:
features=extract_features(labeled_data)

In [41]:
#only for printing features 
features['labels']

[2, 4, 4, 2, 1, 2, 0, 2, 2, 2]

In [141]:
classifier=svm.SVC()

In [142]:
classifier.fit(features['feature'],features['labels'])
with open('svc_bigram_model','w') as f:
    pickle.dump(classifier,f)

In [11]:
sen='work life is bore'
classifier.classify(extract_features(sen.split()))

'neg'

In [21]:
dist = classifier.prob_classify(extract_features(sen.split()))
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931


In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903
